In [1]:
!pip install faker

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ---------------------------- ----------- 1.3/1.8 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 4.8 MB/s eta 0:00:00


In [21]:
import sqlite3
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Connect to SQLite database (or create it)
conn = sqlite3.connect('courier_services5.db')
cursor = conn.cursor()

# Create Tables
cursor.execute('''CREATE TABLE IF NOT EXISTS Packages (
    package_id INTEGER PRIMARY KEY,
    weight REAL,
    length REAL,
    width REAL,
    height REAL,
    content_description TEXT
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Receivers (
    receiver_id INTEGER PRIMARY KEY,
    name TEXT,
    address TEXT,
    contact_number TEXT
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Senders (
    sender_id INTEGER PRIMARY KEY,
    name TEXT,
    address TEXT,
    contact_number TEXT
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Shipments (
    shipment_id INTEGER PRIMARY KEY,
    package_id INTEGER,
    sender_id INTEGER,
    receiver_id INTEGER,
    shipping_method TEXT,
    tracking_number TEXT,
    status TEXT,
    FOREIGN KEY (package_id) REFERENCES Packages(package_id),
    FOREIGN KEY (sender_id) REFERENCES Senders(sender_id),
    FOREIGN KEY (receiver_id) REFERENCES Receivers(receiver_id)
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Claims (
    claim_id INTEGER PRIMARY KEY,
    shipment_id INTEGER,
    claim_status TEXT,
    claim_date TEXT,
    resolution_date TEXT,
    amount_claimed REAL,
    FOREIGN KEY (shipment_id) REFERENCES Shipments(shipment_id)
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS CustomerFeedback (
    feedback_id INTEGER PRIMARY KEY,
    shipment_id INTEGER,
    customer_id INTEGER,
    rating INTEGER,
    comments TEXT,
    feedback_date TEXT,
    FOREIGN KEY (shipment_id) REFERENCES Shipments(shipment_id)
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS DeliveryAttempts (
    attempt_id INTEGER PRIMARY KEY,
    shipment_id INTEGER,
    attempt_date TEXT,
    attempt_status TEXT,
    notes TEXT,
    FOREIGN KEY (shipment_id) REFERENCES Shipments(shipment_id)
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS DeliveryLocations (
    location_id INTEGER PRIMARY KEY,
    shipment_id INTEGER,
    longitude REAL,
    latitude REAL,
    delivery_date TEXT,
    FOREIGN KEY (shipment_id) REFERENCES Shipments(shipment_id)
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS ServiceAreas (
    area_id INTEGER PRIMARY KEY,
    service_area_name TEXT,
    coverage_description TEXT,
    is_active BOOLEAN
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS ShippingRates (
    rate_id INTEGER PRIMARY KEY,
    shipping_method TEXT,
    weight_limit REAL,
    base_price REAL,
    additional_cost_per_kg REAL
)''')

# Commit table creation
conn.commit()

# Helper functions to generate random data
def create_packages(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO Packages (weight, length, width, height, content_description) 
                          VALUES (?, ?, ?, ?, ?)''', 
                          (random.uniform(1, 10), random.uniform(10, 100), random.uniform(10, 100),
                           random.uniform(10, 100), fake.text(max_nb_chars=50)))
    conn.commit()

def create_receivers(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO Receivers (name, address, contact_number) 
                          VALUES (?, ?, ?)''', 
                          (fake.name(), fake.address(), fake.phone_number()))
    conn.commit()

def create_senders(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO Senders (name, address, contact_number) 
                          VALUES (?, ?, ?)''', 
                          (fake.name(), fake.address(), fake.phone_number()))
    conn.commit()

def create_shipments(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO Shipments (package_id, sender_id, receiver_id, shipping_method, tracking_number, status) 
                          VALUES (?, ?, ?, ?, ?, ?)''', 
                          (random.randint(1, num_packages), random.randint(1, num_senders), random.randint(1, num_receivers),
                           random.choice(['Air', 'Ground', 'Sea']), fake.uuid4(), random.choice(['Pending', 'Delivered'])))
    conn.commit()

def create_claims(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO Claims (shipment_id, claim_status, claim_date, resolution_date, amount_claimed) 
                          VALUES (?, ?, ?, ?, ?)''', 
                          (random.randint(1, num_shipments), random.choice(['Open', 'Resolved']), 
                           fake.date_this_year(), fake.date_this_year(), random.uniform(10, 1000)))
    conn.commit()

def create_customer_feedback(n):
    feedback_comments = [
        "The package arrived on time and in great condition.",
        "Delivery was delayed, but the service was responsive.",
        "The packaging was secure, and I appreciate the care taken.",
        "Had a minor issue with the tracking number, but it was resolved quickly.",
        "The delivery person was very polite and professional.",
        "Overall a smooth experience, would use again!",
        "Received the package, but it was damaged. Needs better handling.",
        "Quick delivery but the tracking updates were lacking.",
        "Excellent service! Will definitely recommend.",
        "The delivery was successful, but I would prefer more updates."
    ]
    
    for _ in range(n):
        cursor.execute('''INSERT INTO CustomerFeedback (shipment_id, customer_id, rating, comments, feedback_date) 
                          VALUES (?, ?, ?, ?, ?)''', 
                          (random.randint(1, num_shipments), random.randint(1, num_senders), 
                           random.randint(1, 5), random.choice(feedback_comments), 
                           fake.date_this_year()))
    conn.commit()

def create_delivery_attempts(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO DeliveryAttempts (shipment_id, attempt_date, attempt_status, notes) 
                          VALUES (?, ?, ?, ?)''', 
                          (random.randint(1, num_shipments), fake.date_this_year(), random.choice(['Success', 'Failed']),
                           fake.sentence()))
    conn.commit()

def create_delivery_locations(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO DeliveryLocations (shipment_id, longitude, latitude, delivery_date) 
                          VALUES (?, ?, ?, ?)''', 
                          (random.randint(1, num_shipments), float(fake.longitude()), float(fake.latitude()), fake.date_this_year()))
    conn.commit()

def create_service_areas(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO ServiceAreas (service_area_name, coverage_description, is_active) 
                          VALUES (?, ?, ?)''', 
                          (fake.city(), fake.text(max_nb_chars=100), random.choice([True, False])))
    conn.commit()

def create_shipping_rates(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO ShippingRates (shipping_method, weight_limit, base_price, additional_cost_per_kg) 
                          VALUES (?, ?, ?, ?)''', 
                          (random.choice(['Air', 'Ground', 'Sea']), random.uniform(1, 50), random.uniform(5, 50), random.uniform(1, 5)))
    conn.commit()

# Generate Data for a larger set
num_packages = 1500
num_senders = 1200
num_receivers = 1200
num_shipments = 2500
num_delivery_attempts = 2500
num_delivery_locations = 1500
num_shipping_rates = 20
num_service_areas = 50
num_customer_feedback = 2000
num_claims = 500

create_packages(num_packages)
create_receivers(num_receivers)  # Receivers before Senders
create_senders(num_senders)
create_shipments(num_shipments)
create_claims(num_claims)
create_customer_feedback(num_customer_feedback)
create_delivery_attempts(num_delivery_attempts)
create_delivery_locations(num_delivery_locations)
create_service_areas(num_service_areas)
create_shipping_rates(num_shipping_rates)

# Close the connection
conn.close()


In [22]:
import os
import shutil

# Get the path to the desktop
desktop_path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')  # For Windows

# For MacOS/Linux, uncomment the line below:
# desktop_path = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop')

# Path to the SQLite database file
db_file = 'courier_services5.db'

# Define the target path (your desktop)
target_path = os.path.join(desktop_path, 'courier_services2.db')

# Copy the file to your desktop
shutil.copy(db_file, target_path)

print(f"Database has been copied to your desktop: {target_path}")


Database has been copied to your desktop: C:\Users\chait\Desktop\courier_services2.db


In [23]:
import sqlite3

def export_sqlite_to_sql(db_file, sql_file):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Open the SQL file for writing
    with open(sql_file, 'w') as f:
        # Dump the schema (tables)
        for line in conn.iterdump():
            f.write('%s\n' % line)

    # Close the connection
    conn.close()

# Usage
db_file = 'courier_services5.db'  # Replace with your SQLite database file
sql_file = 'courier_services5.sql'  # Replace with the desired SQL output file name
export_sqlite_to_sql(db_file, sql_file)

print(f"Exported {db_file} to {sql_file} successfully.")


Exported courier_services5.db to courier_services5.sql successfully.
